# CPU Model Runner

Assuming you have models saved at `model_cpu/bin`

## Setup

In [1]:
import sys
print(sys.executable)
print(sys.version)

import warnings
warnings.filterwarnings("ignore")

c:\Python312\python.exe
3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

import os
import time

In [3]:
NUM_THREADS = 2
torch.set_num_threads(NUM_THREADS)         # Affects intra-op parallelism (e.g., GEMM)
torch.set_num_interop_threads(NUM_THREADS) # Affects inter-op parallelism

In [4]:
from model_cpu.model import SimpleCNN

In [5]:
# Define the directory and file path
SAVE_DIR = "model_cpu/bin"

FP32_MODEL_NAME = "CNN-MNIST-CPU-fp32.pt"
FP32_SAVE_PATH = os.path.join(SAVE_DIR, FP32_MODEL_NAME)

INT8_MODEL_NAME = "CNN-MNIST-CPU-int8.pt"
INT8_SAVE_PATH = os.path.join(SAVE_DIR, INT8_MODEL_NAME)

In [6]:
# --- Hyperparameters and Setup ---
DEVICE = torch.device("cpu")
LEARNING_RATE = 0.001
BATCH_SIZE = 64
NUM_EPOCHS = 10
TRAIN_VAL_SPLIT_RATIO = 0.9 # 90% for training, 10% for validation

In [7]:
# --- 2. Load Data and Define DataLoaders ---
def get_dataloaders():
    # Standard transforms for MNIST
    # ToTensor() scales images from [0, 255] to [0.0, 1.0]
    # Normalize() standardizes the data. (0.1307,) and (0.3081,) are
    # the pre-calculated mean and std dev of the MNIST dataset.
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # Download the full training dataset (60,000 images)
    full_train_dataset = datasets.MNIST(
        root='./data', 
        train=True, 
        transform=transform, 
        download=True
    )
    
    # Download the test dataset (10,000 images)
    test_dataset = datasets.MNIST(
        root='./data', 
        train=False, 
        transform=transform, 
        download=True
    )
    
    # Split the full training dataset into training and validation sets
    train_size = int(len(full_train_dataset) * TRAIN_VAL_SPLIT_RATIO)
    val_size = len(full_train_dataset) - train_size
    
    train_subset, val_subset = random_split(
        full_train_dataset, 
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42) # for reproducible splits
    )
    
    print(f"Total training images: {len(train_subset)}")
    print(f"Total validation images: {len(val_subset)}")
    print(f"Total test images: {len(test_dataset)}")
    
    # Create DataLoaders for each set
    train_loader = DataLoader(
        dataset=train_subset, 
        batch_size=BATCH_SIZE, 
        shuffle=True
    )
    val_loader = DataLoader(
        dataset=val_subset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
    )
    test_loader = DataLoader(
        dataset=test_dataset, 
        batch_size=BATCH_SIZE, 
        shuffle=False
    )
    
    return train_loader, val_loader, test_loader

In [8]:
print(f"Using device: {DEVICE}")
    
# Get DataLoaders
train_loader, val_loader, test_loader = get_dataloaders()

# Initialize model, loss function, and optimizer
model = SimpleCNN().to(DEVICE)
criterion = nn.CrossEntropyLoss() # Combines LogSoftmax and NLLLoss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# History lists to store metrics
train_loss_history = []
train_acc_history = []
val_loss_history = []
val_acc_history = []

Using device: cpu
Total training images: 54000
Total validation images: 6000
Total test images: 10000


In [9]:
# --- Helper Functions for Comparison ---
# Helper to print model size
def print_model_size(model, label):
    # Save a temporary file
    torch.save(model.state_dict(), "temp.pt")
    size_mb = os.path.getsize("temp.pt") / (1024 * 1024)
    print(f"Size of {label} model: {size_mb:.2f} MB")
    os.remove("temp.pt")

# Helper to evaluate accuracy
def evaluate_model(model, data_loader, device):
    model.eval()
    model.to(device)
    
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    return 100 * correct / total

# Helper to measure inference speed (Throughput)
def measure_inference_speed(model, data_loader, device):
    model.eval()
    model.to(device)
    
    # Use one batch for a warm-up/test run
    dummy_input = next(iter(data_loader))[0].to(device)
    
    # Warm-up runs
    print("  Running warm-up...")
    for _ in range(10):
        _ = model(dummy_input)
        
    # Measure
    print("  Measuring inference throughput...")
    start_time = time.time()
    total_samples = 0
    
    with torch.no_grad():
        for images, _ in data_loader:
            _ = model(images.to(device))
            # Add the batch size (number of images in this batch)
            total_samples += images.size(0)
            
    end_time = time.time()
    
    total_time = end_time - start_time
    samples_per_second = total_samples / total_time
    
    return samples_per_second

# Helper to get model size in bytes
def get_model_size(model):
    """Saves model state_dict temporarily to get file size."""
    # Save a temporary file
    torch.save(model.state_dict(), "temp_size_calc.pt")
    size_bytes = os.path.getsize("temp_size_calc.pt")
    os.remove("temp_size_calc.pt")
    return size_bytes

## Run

In [10]:
# --- Run the Comparison ---
print("\n--- Comparing FP32 and INT8 Models ---")

# (Ensure test_loader and val_loader are loaded)
if 'test_loader' not in locals():
    print("Loading data for evaluation...")
    _, val_loader, test_loader = get_dataloaders()
    
# --- 1. Load the original FP32 model ---
print(f"Loading FP32 model from '{FP32_SAVE_PATH}'...")
fp32_model = SimpleCNN().to(DEVICE)
fp32_model.load_state_dict(
    torch.load(FP32_SAVE_PATH), 
    strict=False # For quant/dequant stubs
)
fp32_model.eval()

# --- 2. Load the saved INT8 model ---
print(f"Loading INT8 model from '{INT8_SAVE_PATH}'...")
# To load a quantized model, you MUST recreate the same
# fused and quantized structure *before* loading the state_dict.
int8_model_to_load = SimpleCNN().to(DEVICE)
int8_model_to_load.eval()
int8_model_to_load.fuse_model() # Fuse first
int8_model_to_load.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(int8_model_to_load, inplace=True)
torch.quantization.convert(int8_model_to_load, inplace=True) 

# Now load the saved INT8 state dict
int8_model_to_load.load_state_dict(
    torch.load(INT8_SAVE_PATH)
)
int8_model = int8_model_to_load
print("Models loaded.")

# --- 3. Compare Model Size ---
print("\n--- Model Size Comparison ---")
print_model_size(fp32_model, "FP32")
print_model_size(int8_model, "INT8")

# --- 4. Compare Accuracy ---
print("\n--- Accuracy Comparison (on Test Set) ---")
fp32_accuracy = evaluate_model(fp32_model, test_loader, DEVICE)
print(f"FP32 Model Accuracy: {fp32_accuracy:.2f}%")

int8_accuracy = evaluate_model(int8_model, test_loader, DEVICE)
print(f"INT8 Model Accuracy: {int8_accuracy:.2f}%")

# --- 5. Compare Inference Speed ---
print("\n--- Inference Speed Comparison (on Validation Set) ---")
print("Measuring FP32 model...")
fp32_throughput = measure_inference_speed(fp32_model, val_loader, DEVICE)
print(f"-> FP32 Throughput: {fp32_throughput:.2f} samples/sec")

print("\nMeasuring INT8 model...")
int8_throughput = measure_inference_speed(int8_model, val_loader, DEVICE)
print(f"-> INT8 Throughput: {int8_throughput:.2f} samples/sec")

# --- 6. Summary ---
print("\n--- Summary ---")
fp32_size_bytes = get_model_size(fp32_model)
int8_size_bytes = get_model_size(int8_model)

print(f"Size Reduction: ~{((1 - (int8_size_bytes / fp32_size_bytes)) * 100):.1f}%")
# Speedup calculation: Faster / Slower
print(f"Speedup: {int8_throughput / fp32_throughput:.2f}x")
print(f"Accuracy Drop: {fp32_accuracy - int8_accuracy:.2f}%")


--- Comparing FP32 and INT8 Models ---
Loading FP32 model from 'model_cpu/bin\CNN-MNIST-CPU-fp32.pt'...
Loading INT8 model from 'model_cpu/bin\CNN-MNIST-CPU-int8.pt'...
Models loaded.

--- Model Size Comparison ---
Size of FP32 model: 1.74 MB
Size of INT8 model: 0.45 MB

--- Accuracy Comparison (on Test Set) ---
FP32 Model Accuracy: 99.17%
INT8 Model Accuracy: 99.16%

--- Inference Speed Comparison (on Validation Set) ---
Measuring FP32 model...
  Running warm-up...
  Measuring inference throughput...
-> FP32 Throughput: 1429.40 samples/sec

Measuring INT8 model...
  Running warm-up...
  Measuring inference throughput...
-> INT8 Throughput: 1938.85 samples/sec

--- Summary ---
Size Reduction: ~74.3%
Speedup: 1.36x
Accuracy Drop: 0.01%
